In [2]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import torch.nn.functional as F
import numpy as np
# 学习率：控制梯度更新步长，过大易震荡，过小易收敛慢
learning_rate = 1e-4
# Dropout保留率：保留神经元的概率；PyTorch 的 Dropout 使用的是丢弃概率 p，因此后面用 (1 - 保留率)
keep_prob_rate = 0.7
# 训练轮数：完整遍历训练集的次数
max_epoch = 3
# 每批样本数量，影响显存占用与梯度估计稳定性
BATCH_SIZE = 50

DOWNLOAD_MNIST = False
# 如果不存在本地 ./mnist/ 目录或目录为空，则触发下载
if not(os.path.exists('./mnist/')) or not os.listdir('./mnist/'):
    # not mnist dir or mnist is empty dir
    DOWNLOAD_MNIST = True


# 训练集：ToTensor() 将 [0,255] 像素归一化到 [0,1] 并转为张量 (C,H,W)
train_data = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=torchvision.transforms.ToTensor(), download=DOWNLOAD_MNIST,)
# DataLoader 负责按批次加载、打乱样本以减少梯度方差
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

# 测试集：不做 transform，下面手动归一化
test_data = torchvision.datasets.MNIST(root='./mnist/', train=False)
# 取测试集前500张图片，转换为浮点并归一化到[0,1]；增加通道维度为 [batch, 1, 28, 28]
test_x = test_data.test_data.unsqueeze(1).float()[:500] / 255.  # 形状：[500, 1, 28, 28]
# 注意：新版本 torchvision 中属性已更名为 data/targets，这里沿用旧写法
test_y = test_data.test_labels[:500].numpy()

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            # 输入 [B,1,28,28] -> Conv7x7 保持 28x28 -> MaxPool2d(2) -> [B,32,14,14]
            nn.Conv2d(
                # 7x7 卷积核；输入通道=1（灰度图），输出通道=32；步幅=1
                # padding = (kernel_size - 1) // 2 = 3，保持特征图尺寸不变（28x28）
                in_channels=1,
                out_channels=32,
                kernel_size=7,
                stride=1,
                padding=3
            ),
            nn.ReLU(),        # 激活函数
            nn.MaxPool2d(2)   # 2x2 最大池化，尺寸从 28x28 -> 14x14
        )
        self.conv2 = nn.Sequential(
            # 输入 [B,32,14,14] -> Conv5x5 保持 14x14 -> MaxPool2d(2) -> [B,64,7,7]
            nn.Conv2d(
                # 5x5 卷积核；输入通道=32，输出通道=64；步幅=1；padding=2 保持尺寸（14x14）
                in_channels=32,
                out_channels=64,
                kernel_size=5,
                stride=1,
                padding=2
            ),
            nn.ReLU(),        # 激活函数
            nn.MaxPool2d(2)   # 2x2 最大池化，尺寸从 14x14 -> 7x7
        )
        # Flatten 后维度为 64*7*7=3136；映射到较大的隐藏层以提升表达能力
        self.out1 = nn.Linear(7*7*64, 1024, bias=True)   # 全连接层1：输入64*7*7，输出1024
        self.dropout = nn.Dropout(p=1 - keep_prob_rate)  # PyTorch 的 p 是丢弃概率；仅在 model.train() 时生效
        self.out2 = nn.Linear(1024,10,bias=True)



    def forward(self, x):
        x = self.conv1(x)  # -> [B,32,14,14]
        x = self.conv2(x)  # -> [B,64,7,7]
        x = x.view(x.size(0), -1)  # 展平：形状 [batch, 64, 7, 7] -> [batch, 64*7*7]
        out1 = self.out1(x)
        out1 = F.relu(out1)
        out1 = self.dropout(out1)
        out2 = self.out2(out1)           # 分类层输出 logits（未归一化分数）
        output = F.softmax(out2, dim=1)  # 指定类别维度做 softmax，得到每类概率；训练中更常直接用 logits 配合 CrossEntropyLoss
        return output


def test(cnn):
    # 评估模式：禁用Dropout等训练行为
    cnn.eval()
    with torch.no_grad():
        y_pre = cnn(test_x)
        # 取每一行的最大概率对应类别
        _, pre_index = torch.max(y_pre, 1)
        prediction = pre_index.view(-1).cpu().numpy()
    correct = np.sum(prediction == test_y)
    return correct / 500.0


def train(cnn):
    # Adam 优化器：自适应学习率，通常在视觉任务上表现稳健
    optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)
    # 交叉熵损失：期望输入为 logits（未做 softmax）；本实现传入概率不会报错但并非最佳实践
    loss_func = nn.CrossEntropyLoss()
    for epoch in range(max_epoch):
        cnn.train()  # 训练模式：启用 Dropout 等行为
        for step, (x, y) in enumerate(train_loader):
            output = cnn(x)  # 前向计算得到类别概率（本实现显式 softmax；更佳做法是返回 logits）
            loss = loss_func(output, y)  # 计算当前批次的交叉熵损失
            optimizer.zero_grad()  # 清空上一轮的梯度缓存
            loss.backward()  # 反向传播计算梯度
            optimizer.step()  # 按优化器规则更新参数
            
            if step != 0 and step % 20 == 0:
                print("=" * 10, step, "=" * 5, "=" * 5, "test accuracy is ", test(cnn), "=" * 10)

if __name__ == '__main__':
    cnn = CNN()
    train(cnn)




100.0%
100.0%
100.0%
100.0%
d:\code\python\deeplearning\.venv\Lib\site-packages\torchvision\datasets\mnist.py:81: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
d:\code\python\deeplearning\.venv\Lib\site-packages\torchvision\datasets\mnist.py:71: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


========== 20 ===== ===== test accuracy is  0.25 ==========
========== 40 ===== ===== test accuracy is  0.442 ==========
========== 60 ===== ===== test accuracy is  0.604 ==========
========== 80 ===== ===== test accuracy is  0.614 ==========
========== 100 ===== ===== test accuracy is  0.646 ==========
========== 120 ===== ===== test accuracy is  0.73 ==========
========== 140 ===== ===== test accuracy is  0.772 ==========
========== 160 ===== ===== test accuracy is  0.852 ==========
========== 180 ===== ===== test accuracy is  0.878 ==========
========== 200 ===== ===== test accuracy is  0.872 ==========
========== 220 ===== ===== test accuracy is  0.884 ==========
========== 240 ===== ===== test accuracy is  0.892 ==========
========== 260 ===== ===== test accuracy is  0.898 ==========
========== 280 ===== ===== test accuracy is  0.902 ==========
========== 300 ===== ===== test accuracy is  0.898 ==========
========== 320 ===== ===== test accuracy is  0.908 ==========
========== 340